# Mode d'emploi
Ce script permet de changer le SCR de points décrits par leurs coordonnées X, Y dans un fichier texte délimité. Le code EPSG de la projection des données d'entrée doit être connu.

0. Créer sur le serveur un répertoire input/ et un répertoire output/

1. Il est possible de reprojeter un unique fichier ou bien une série de fichiers placés dans une archive ZIP. Uploader le fichier ou l'archive dans le répertoire input/

2. Exécuter les cellules correspondant à la section "Eléments communs"

**Pour reprojeter un unique fichier** : se reporter à la section "reprojeter un unique tableau"
3. Dans le champ "Paramètres", indiquer le nom du fichier, le code de l'EPSG source et le code EPSG de destination
4. Exécuter ensuite les cellules correspondant à cette section
5. Télécharger le fichier reprojeté depuis le répertoire output/

**Pour reprojeter tous les fichiers placés dans une archive** : se reporter à la section "reprojeter plusieurs tableaux au sein d'une archive"
3. Dans le champ "Paramètres", indiquer le nom de l'archive, le code de l'EPSG source et le code EPSG de destination
4. Exécuter ensuite les cellules correspondant à cette section
5. Télécharger l'archive ZIP depuis le répertoire output/


# Outil de reprojection
## Eléments communs

In [ ]:
'''
Conversion des colonnes X et Y d'un tableau vers un SCR spécifié (d'après son EPSG)
'''
import os
import geopandas as gpd
import pandas as pd
import shapely.geometry as sg
import matplotlib.pyplot as plt

In [ ]:
# Fonction de reprojection d'un tableau contenant des colonnes X et Y
def reproj_tbl(df, epsg_src, epsg_cbl):
    # Les colonnes sont enregistrées pour permettre leur restitution dans le même ordre
    cols = df.columns.tolist()
    
    gdf = gpd.GeoDataFrame(df.drop(['X', 'Y'], axis=1),
                      crs={'init': 'epsg:'+str(epsg_src)},
                      geometry=[sg.Point(xy) for xy in zip(df.X, df.Y)])
    
    # Reprojection
    reproj_gdf = gdf.to_crs(epsg=str(epsg_cbl))
    
    # Reconversion en tableau (DataFrame) avec des colonnes X et Y
    out_df = pd.DataFrame(reproj_gdf.drop(["geometry"], axis=1))
    out_df["X"] = reproj_gdf.geometry.x
    out_df["Y"] = reproj_gdf.geometry.y
    
    return(out_df[cols])

# Fonction de reprojection d'un fichier délimité contenant des colonnes X et Y
def reproj_delim(input_tbl_fpath, output_tbl_path, tbl_delim, epsg_src, epsg_cbl):
    # Lecture des données d'entrée et conversion en GeoDataFrame
    df = pd.read_csv(input_tbl_fpath, delimiter=tbl_delim)
    
    # Calcul du tableau reprojeté et écriture du fichier délimité
    reproj_tbl(df, epsg_src, epsg_cbl).to_csv(output_tbl_fpath, sep=tbl_delim, ,index=False)

## Reprojeter un unique tableau

Le tableau à reprojeter doit :
* posséder un en-tête (cohérent avec le contenu) ;
* posséder une colonne X et une colonne Y.


In [ ]:
# Paramètres
input_dir = "input"
input_tbl_name = "NRA_01"
tbl_ext = ".csv"
tbl_delim = ";"

epsg_src=7421
epsg_cbl=2154

output_dir = "output"
output_tbl_name = input_tbl_name

In [ ]:
# Calcul du chemin des fichiers d'entrée et de sortie
input_tbl_fpath = input_dir + "/" + input_tbl_name + tbl_ext
output_tbl_fpath = output_dir + "/" + output_tbl_name + tbl_ext

In [ ]:
reproj_delim(input_tbl_fpath, output_tbl_fpath, tbl_delim, epsg_src, epsg_cbl)

## Reprojeter plusieurs tableaux au sein d'une archive
Les tableaux doivent être dans une archive ZIP, au 2e niveau d'arborescence. Par exemple :
* dir1/
  * dir1/foo1.csv,
  * dir1/bar1.csv ;
* dir2/
  * dir2/foo2.csv,
  * dir2/bar2.csv.

D'autre part tous les tableaux contenus au 2e niveau d'arborescence doivent :
* posséder un en-tête (cohérent avec le contenu) ;
* posséder une colonne X et une colonne Y ;
* exprimées dans le même système de coordonnées de référence (CRS).

**Remarque** : dans ce cas, les répertoires du premier niveau de l'arborescence du ZIP sont décompressés un à un ; il est donc nécessaire de disposer sur la machine d'un espace de stockage équivalent à deux fois la taille compressée de l'archive (entrée + sortie) plus deux fois la taille décompressée d'un répertoire du premier niveau (tableaux originaux et reprojetés).

In [ ]:
# Paramètres
input_dir = "input"
input_zip_name = "NRA" # sans l'extension .zip
tbl_ext = ".csv"
tbl_delim = ";"

epsg_src=7421
epsg_cbl=2154

output_dir = "output"
output_zip_name = input_zip_name + "_EPSG" + str(epsg_cbl)

In [ ]:
# Calcul du chemin des fichiers d'entrée et de sortie
input_zip_fpath = input_dir + "/" + input_zip_name + ".zip"
output_zip_fpath = "../../" + output_dir + "/" + output_zip_name + ".zip"

In [ ]:
# Liste des répertoires contenus dans le zip
list_dir_names = !unzip -Z1 $input_zip_fpath "*/"

print("Contenu de l'archive :")
for dir_name in list_dir_names:
    print(dir_name)

In [ ]:
os.makedirs("temp/in")
os.makedirs("temp/out")

for dir_name in list_dir_names:
    print("Traitement des shapefiles du répertoire : " + dir_name)
    
    # décompression du répertoire de l'archive dans temp/in
    dir_path = "temp/in/" + dir_name
    to_unzip = dir_name + "*"
    !unzip -q $input_zip_fpath $to_unzip -d "temp/in/"
    
    # création du répertoire temporaire de sortie
    dir_reprojete = "temp/out/" + dir_name
    os.makedirs("temp/out/" + dir_name)
    
    # parcours des répertoires et réécriture
    for input_file in os.listdir("temp/in/" + dir_name):
        if input_file.endswith(tbl_ext.lower()) | input_file.endswith(tbl_ext.upper()):
            
            input_tbl_fpath = "temp/in/" + dir_name + input_file
            output_tbl_fpath = "temp/out/" + dir_name + input_file
            
            # Reprojection et écriture du fichier délimité reprojeté
            reproj_delim(input_tbl_fpath, output_tbl_fpath, tbl_delim, epsg_src, epsg_cbl)
        
    # recompression des fichiers de sortie dans l'archive résultat
    # N.B. qu'il est nécessaire de se déplacer sinon l'archive contient l'arborescence temp/out/etc.
    %cd -q temp/out/
    print("Compression de " + dir_reprojete)
    !zip -rq $output_zip_fpath $dir_name
    %cd -q ../../
    
    # suppression des fichiers d'entrée et de sortie
    !rm -r $dir_path
    !rm -r $dir_reprojete
    
# Supprimer les fichiers temporaires
!rm -r "temp/in"
!rm -r "temp/out"

# Utilitaires
## Tracer les points

In [ ]:
%matplotlib inline
%pylab inline
df.plot.scatter(x='X',y='Y')

In [ ]:
out_df.plot.scatter(x='X',y='Y')

## Supprimer les fichiers temporaires
(en cas d'erreur lors de l'exécution...)

In [ ]:
# Supprimer les fichiers temporaires
!rm -r "temp/in"
!rm -r "temp/out"